In [2]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage
from langchain.schema import AIMessage
from langchain.schema import SystemMessage
from langchain.schema import BaseOutputParser

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate

from langchain.callbacks import StreamingStdOutCallbackHandler


In [2]:
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct"
)

llm.predict("How many planets are there?")

'\n\nAs of 2021, there are eight officially recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, some scientists argue that there may be a ninth planet, often referred to as Planet Nine, located beyond Neptune. '

In [3]:
chat = ChatOpenAI(
    # temperature=0.1
)

chat.predict("How many planets are there?")

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in {language}.",
    ),
    AIMessage(content="Ciao, mi chiamo {name}!"),
    HumanMessage(
        content="What is the distance between {country_a} and {country_b}. Also, what is your name?",
    ),
]

chat = ChatOpenAI()
chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi. Il mio nome è Assistente di Lingue! Come posso aiutarti oggi?')

In [5]:
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat = ChatOpenAI()
chat.predict(prompt)

'The distance between Mexico City, Mexico and Bangkok, Thailand is approximately 16,625 kilometers (10,324 miles) when measured in a straight line.'

In [12]:
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

chat = ChatOpenAI(
    temperature=0.1
)

chain = template | chat | CommaOutputParser()
chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [16]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ""),
        ("human", "I want to cook {cuisine} food."),
    ]
)
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)
veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian"})

That's great! Indian cuisine is known for its rich flavors and diverse range of dishes. Here are a few popular Indian recipes you can try:

1. Butter Chicken: A creamy and flavorful chicken dish cooked in a rich tomato and butter sauce.

2. Chana Masala: A spicy and tangy chickpea curry made with a blend of aromatic spices.

3. Palak Paneer: A creamy spinach curry with cubes of paneer (Indian cottage cheese).

4. Biryani: A fragrant and flavorful rice dish cooked with meat, vegetables, and a blend of spices.

5. Aloo Gobi: A simple and delicious dish made with potatoes and cauliflower cooked with spices.

6. Dal Tadka: A comforting lentil curry tempered with aromatic spices.

These are just a few examples, but there are countless other Indian dishes you can explore. Enjoy your cooking!Thank you for sharing these delicious Indian recipes! As a vegetarian chef specializing in traditional recipes, I can offer some alternative ingredients for a vegetarian twist on these dishes:

1. Butter 

AIMessageChunk(content='Thank you for sharing these delicious Indian recipes! As a vegetarian chef specializing in traditional recipes, I can offer some alternative ingredients for a vegetarian twist on these dishes:\n\n1. Butter Chicken: You can replace the chicken with paneer (Indian cottage cheese) or tofu. Marinate the paneer or tofu in a mixture of yogurt, lemon juice, and spices before cooking it in the creamy tomato and butter sauce.\n\n2. Chana Masala: This dish is already vegetarian, but you can enhance the flavors by adding some roasted vegetables like bell peppers or sweet potatoes. You can also add some coconut milk for a creamier texture.\n\n3. Palak Paneer: Instead of paneer, you can use tofu or boiled and cubed potatoes. Blend the spinach with some cashew cream or coconut milk for a creamy texture.\n\n4. Biryani: For a vegetarian version, you can use a mix of vegetables like carrots, peas, potatoes, and cauliflower. You can also add paneer or tofu for protein. Make sure 